## Descarga y procesamiento de imagenes DEM y LULC para modelo de inundaciones

Este notebook presenta:

1- codigo para descarga de imagenes DEM-SRTM desde la plataforma Google Earth Engine (GEE) y su posterior procesamiento para la conformacion del dataset a utilizar en el modelo de susceptibilidad a inundaciones. Es importante aclarar que para poder correr este notebook se debe estar registrado en GEE; si es necesario registrarse ingresar en https://earthengine.google.com/ y seguir los pasos alli detallados.

2- procesamiento del mapa de Cobertura y Uso de Suelo 2021 (LULC 2021) elaborado por ESRI y descargados desde https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2.

## 1 Descarga de DEM

## 1.1 Importacion de librerias y definicion de variables

In [ ]:
import ee
import geemap
import os
from geemap import geojson_to_ee, ee_to_geojson
from osgeo import gdal
import numpy as np
import shutil

In [ ]:
#bucket con archivo vectorial del area de interes para modelo de inundaciones
BUCKET_AOI = "gs://dym-indunor-temp/immap/final/aux_data/floods_suscp/aoi/*.geojson" 

#bucket donde se guardara la imagen DEM a descargas desde GEE
BUCKET_SRTM = "gs://dym-indunor-temp/immap/final/aux_data/floods_suscp/srtm/*.tif"

#creamos las carpeta a donde descargaremos los archivos desde bucket
BASE_PATH = "../"
PATH_AOI = f"{BASE_PATH}/aoi_flood/"
SRTM_PATH = f"{BASE_PATH}/srtm/"

## 1.2. Autenticacion e Inicio de Google Earth Engine

Como primer paso se debe autenticar el ingreso a los servidores de GEE y dar inicio. Una vez que se corra la celda a continuacion, se le va a solicitar que ingrese a su cuenta de Google para dar acceso. Siga las instrucciones, copie el codigo de autorizacion y peguelo en la celda vacia.

Una vez que las credenciales de autenticacion han sido guardadas en el sistema, solo es necesario importar la API de Python para GEE e inicilizarlo. Para mas informacion, dirigase a https://developers.google.com/earth-engine/guides/python_install.

In [ ]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

## 1.3. Descarga de archivos desde bucket

In [ ]:
!mkdir -p $PATH_AOI   #crea la carpeta
!gsutil -m cp -r $BUCKET_AOI   $PATH_AOI # descarga de imagenes

## 1.4. Descarga de imagenes DEM

In [ ]:
for file in os.listdir(PATH_AOI):
    aoi = (geojson_to_ee(f'{PATH_AOI}{file}')).geometry()

    srtm = ee.Image('USGS/SRTMGL1_003').clip(aoi)
    
    # descarga de imagen a bucket
    task = ee.batch.Export.image.toCloudStorage(
            image = srtm,
            bucket = 'dym-indunor-temp/',
            fileNamePrefix = f"immap/final/aux_data/floods_suscp/srtm/dem_srtm_500m_32618",
            maxPixels = 8030040147504,
            scale = 500,
            region = aoi
            crs='EPSG:32618')
    
    task.start()

In [ ]:
# DESCARGO DEL BUCKET EL ARCHIVO DEM DESCARGADO DESDE GEE
!mkdir -p $SRTM_PATH   #crea la carpeta
!gsutil -m cp -r $BUCKET_SRTM   $SRTM_PATH # descarga de imagenes

## 1.5. Preparacion del DEM

In [ ]:
# setear los valores no dato como 'nan'
for file in os.listdir(SRTM_PATH):
    filename = f'{SRTM_PATH}{file}'
    dem = gdal.Open(filename)
    cols = dem.RasterXSize 
    rows = dem.RasterYSize 
    proj = dem.GetProjection()
    geotr = dem.GetGeoTransform();
    dem_raster = dem.GetRasterBand(1)
    noDataVal = dem_raster.GetNoDataValue()
    
    dem_array = dem.GetRasterBand(1).ReadAsArray(0,0,cols,rows).astype(float) 
    dem_nodata = dem_array
    dem_nodata[np.where(dem_array == noDataVal)] = np.nan
    
    #save new file
    driver = gdal.GetDriverByName('GTiff')
    file = driver.Create(f'{SRTM_PATH}dem_srtm_500m_32618_final.tif', cols, rows, 1, gdal.GDT_Float32)
    file.GetRasterBand(1).WriteArray(dem_nodata)
    file.GetRasterBand(1).SetNoDataValue(np.nan)

    # spatial ref system
    file.SetProjection(proj)
    file.SetGeoTransform(geotr)
    file.FlushCache()

--------------------------------------------------------------------------------------------------------------------------------

## 2. Procesamiento LULC para modelo inundaciones

## 2.1. Importacion de librerias y definicion de variables

In [ ]:
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
from os import remove
import subprocess
from os import remove
import geopandas as gpd

In [ ]:
#bucket con los archivos a utilizar
#LULC
BUCKET_LULC = "gs://dym-indunor-temp/immap/final/aux_data/inf_settl/LULC/*.tif"

#creamos las carpetas a donde descargaremos los archivos desde el bucket
BASE_PATH = "../"
PATH_LULC_FILES = f"{BASE_PATH}/LULC_files/"

#PARAMETROS GENERALES
aoi = 'colombia'
prj = "32618"

## 2.2. Descarga de archivos desde bucket

In [ ]:
#LULC
!mkdir -p $PATH_LULC_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_LULC   $PATH_LULC_FILES # descarga de archivos

## 2.3. Generar LULC 500m para AOI

In [ ]:
# unificar la proyeccion de los archivos LULC
for file in os.listdir(PATH_LULC_FILES):
    if file.endswith('.tif'):
        ds = gdal.Open(f'{PATH_LULC_FILES}{file}')
        output_file = f'{PATH_LULC_FILES}{file[0:-4]}_epsg{prj}.vrt'
        dsReprj = gdal.Warp(output_file, ds, dstSRS=f'EPSG:{prj}')
        ds = None
        dsReprj = None

#resample to 500m
for file in os.listdir(PATH_LULC_FILES):
    if file.endswith('epsg32618.vrt'):
        infile_resample = f'{PATH_LULC_FILES}{file}'
        outfile_resample = f'{PATH_LULC_FILES}{file[0:-4]}_500m.tif'
        ds_subset = gdal.Open(infile_resample)
        dsResampl = gdal.Warp(outfile_resample, ds_subset, xRes = 500, yRes = 500)
        dsResampl = None
        
#mosaico
search_criteria = f'{PATH_LULC_FILES}*_500m.tif'
files_to_mosaic = glob.glob(os.path.join((Path(PATH_LULC_FILES)), search_criteria))
g = gdal.Warp(f'{PATH_LULC_FILES}lulc2021_mosaic_500m.tif',files_to_mosaic, format="GTiff", options=["TILED=YES"])
g = None
del g

#subset by aoi
for file in os.listdir(PATH_AOI):
    gdf = f'{PATH_AOI}{file}'
    infile_subset = f'{PATH_LULC_FILES}lulc2021_mosaic_500m.tif'
    outfile_subset = f'{PATH_LULC_FILES}lulc2021_aoi_flood.tif'
    dataset_subset = gdal.Open(infile_subset)

    ds_recorte = gdal.Warp(outfile_subset, dataset_subset , cutlineDSName = gdf, cropToCutline = True, dstNodata = np.nan)
    ds_recorte = None

#set no data as nan
raster = gdal.Open(f'{PATH_LULC_FILES}lulc2021_aoi_flood.tif')
noDataVal = raster.GetRasterBand(1).GetNoDataValue() 
array = raster.GetRasterBand(1).ReadAsArray(0,0,raster.RasterXSize,raster.RasterYSize).astype(float) 
reclass = array
reclass[np.where(array == noDataVal)] = np.nan
    
#save new file
driver = gdal.GetDriverByName('GTiff')
file = driver.Create(f'{PATH_LULC_FILES}lulc2021_500m_final.tif', raster.RasterXSize,raster.RasterYSize, 1, gdal.GDT_Float32)
file.GetRasterBand(1).WriteArray(reclass)
file.GetRasterBand(1).SetNoDataValue(np.nan)

# spatial ref system
file.SetProjection(raster.GetProjection())
file.SetGeoTransform(raster.GetGeoTransform())
file.FlushCache()

--------------------------------------------------------------------------------------------------------------------------------

## 3. Creacion del dataset de inundaciones

In [ ]:
from osgeo import gdal, osr
import os
import subprocess
from pathlib import Path

In [ ]:
DATASET_PATH = f"{BASE_PATH}/dataset/"
!mkdir -p $DATASET_PATH 

In [ ]:
#movemos los archivos generados para el dataset a la carpeta de dataset
#altitud
shutil.move(f'{SRTM_PATH}dem_srtm_500m_32618_final.tif', DATASET_PATH)

#lulc
shutil.move(f'{PATH_LULC_FILES}lulc2021_500m_final.tif', DATASET_PATH)

In [ ]:
#creacion dataset final
os.chdir(DATASET_PATH)
files = os.listdir()

file_txt = 'files.txt'

textfile = open(((Path(DATASET_PATH)/file_txt)), "w")
for element in files:
    textfile.write(element + "\n")
textfile.close()

cmd = f'gdalbuildvrt -separate {DATASET_PATH}dataset.vrt -input_file_list {file_txt}'
subprocess.run(cmd, shell=True)

cmd2= f'gdalinfo dataset.vrt'
subprocess.run(cmd2, shell=True)

#reproyeccion a EPSG 4326
ds_rp = gdal.Open('dataset.vrt')
output_file = f'{DATASET_PATH}dataset_final.tif'

gdal.UseExceptions()
dsReprj = gdal.Warp(output_file, ds_rp, dstSRS='EPSG:4326')
ds_rp = None
dsReprj = None